In [1]:
from src.lobbyingDataPage import *
from src.lobbyingScraper import *

In [2]:
def convert_html_list(html_list):
    page_list = []
    for i, html in enumerate(html_list):
        print(f'converting html index {i} into data page')
        if 'Lobbyist Entity' in str(html):
        #    page_list.append(EntityDataPage(html))
            pass
        else:
            page_list.append(LobbyistDataPage(html))
    return page_list

In [3]:
def extract_and_save(html_list):
    page_list = convert_html_list(html_list)
    for page in page_list:
        page.save()

def pull_data(url):
    headers={"User-Agent": "Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148"}
    result = requests.get(url, headers=headers)
    result.raise_for_status()
    return result.content

def download_html_list(url_list):
    html_list = []
    for url in url_list:
        print("Pulling data from " + url)
        html_list.append(pull_data(url))
    return html_list

def save_data_from_url_list(url_list):
    disclosure_links = extract_and_save(download_html_list(url_list))
    html_list = download_html_list(disclosure_links)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def extract_client_links(year):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://www.sec.state.ma.us/LobbyistPublicSearch/Default.aspx'

    driver.get(url)

    driver.find_element('id','ContentPlaceHolder1_rdbSearchByType').click()
    select = Select(driver.find_element(By.CLASS_NAME,'p3'))

    select.select_by_value(year)
    Select(driver.find_element('id','ContentPlaceHolder1_ucSearchCriteriaByType_drpType')).select_by_value('L')
    driver.find_element('id','ContentPlaceHolder1_btnSearch').click()

    find_table = driver.find_element(By.ID,'ContentPlaceHolder1_ucSearchResultByTypeAndCategory_grdvSearchResultByTypeAndCategory')
    links = find_table.find_elements(By.TAG_NAME,'a')
    links_list = [l.get_attribute('href') for l in links if str(l.get_attribute('href')).startswith('javascript') == False]
    driver.quit()
    return links_list

def extract_disclosures(list_of_links):
    disclosure_reports = []

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    for link in list_of_links:
    # print(link)
        driver.get(link)
        all_links = driver.find_elements(By.CLASS_NAME,'BlueLinks')
        disclosure_links = [l.get_attribute('href') for l in all_links if 'CompleteDisclosure' in l.get_attribute('href')]
        for dl in disclosure_links:
            disclosure_reports.append(dl)
    driver.quit()

    return disclosure_reports


In [5]:
import pickle
with open("tests/testfiles/2005html.pkl", "rb") as f:
    html05 = pickle.load(f)
with open("tests/testfiles/2020html.pkl", "rb") as f:
    html20 = pickle.load(f)
test_htmls = html05.__add__(html20)

In [6]:
pages = convert_html_list(html20)


converting html index 0 into data page
converting html index 1 into data page
converting html index 2 into data page
converting html index 3 into data page
converting html index 4 into data page
converting html index 5 into data page
converting html index 6 into data page
converting html index 7 into data page
converting html index 8 into data page
converting html index 9 into data page
converting html index 10 into data page
converting html index 11 into data page
converting html index 12 into data page
converting html index 13 into data page
converting html index 14 into data page
converting html index 15 into data page
converting html index 16 into data page
converting html index 17 into data page
converting html index 18 into data page
converting html index 19 into data page
converting html index 20 into data page
converting html index 21 into data page
converting html index 22 into data page
converting html index 23 into data page
converting html index 24 into data page
converting

In [9]:
len([page for page in pages if len(page.tables.keys()) > 1])

114

In [36]:
str(self1.source_name[1])

'Brendan  Scott  Abel'

In [15]:
from IPython.display import HTML

HTML(str(pages[0].soup.html))

In [15]:
from bs4 import BeautifulSoup as bs
import pandas as pd
soup = bs(test_htmls[171],'html.parser')
some_tables = soup.find_all('tr', {'style': 'vertical-align: top'})
table = some_tables[0]
dfs = pd.read_html(soup.prettify())


In [16]:
header = dfs[5].transpose()
header.columns = header.iloc[0]
header

,Authorizing Officer name,Title,Business name,Address,"City, state, zip code",Country,Phone,NaN
0,Authorizing Officer name,Title,Business name,Address,"City, state, zip code",Country,Phone,NaN
1,Jess Beeson Tocco,CEO,A10 Associates,214 Commercial St.,"Malden, MA 02148",USA,508-864-7629,NaN


In [57]:
table = dfs[5][0:7].transpose()
table.columns = table.iloc[0]
table = table[1:]

In [58]:
header = pd.DataFrame(columns=['Authorizing Officer name','Lobbyist name','Title','Business name','Address','City, state, zip code','Country','Agent type','Phone'])
header.append(table)

C:\Users\geekc\AppData\Local\Temp\ipykernel_16600\1097890747.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  header.append(table)


,Authorizing Officer name,Lobbyist name,Title,Business name,Address,"City, state, zip code",Country,Agent type,Phone
1,Chester G. Atkins,NaN,NaN,"ADS Ventures, Inc.","100 Cambridge Street, Suite 1301","Boston, MA 02114",US,NaN,617-557-7442


In [257]:
good_dfs = [df for df in dfs if str(type(df.columns))=="<class 'pandas.core.indexes.base.Index'>" or 'Client' in str(df) or 'Lobbyist' in str(df)]

In [281]:
good_dfs[-3]

,0
0,Lobbyist: John P Burke


In [256]:
for df in dfs:
    if 'Client' in str(df) or 'Lobbyist' in str(df):
        print(str(df))

                                                   0
0  Lobbyist Public Search  Records prior to regis...
                                                   0   1
0                             Lobbyist Public Search NaN
1  Records prior to registration year 2005 are no... NaN
                                                   0
0                       Disclosure reporting details
1                      Back to search detail summary
2  Lobbyist Entity disclosure reporting period  :...
3  Authorizing Officer name  John  Bartley  Title...
4  Client Compensation  Name  Amount  Penn Nation...
                                                   0
0  Client Compensation  Name  Amount  Penn Nation...
1                         Compensation/Salaries Paid
2  Lobbyist or Entity name  Salaries  John C Bart...
3                                 Operating Expenses
4  Date  Recipient  Type of expense  Amount  12/3...
                                                   0
0                                C

,House / Senate,Bill Number or Agency Name,Bill title or activity,Agent position,Amount,Direct business association
0,House Bill,1155.0,An Act relative to parity of treatment with su...,Neutral,$0.00,NaN
1,Senate Bill,2796.0,An Act Putting Patients First,Neutral,$0.00,NaN
2,House Bill,1716.0,An Act to promote high value and evidence-base...,Neutral,$0.00,NaN
3,Senate Bill,2864.0,An Act relative to improving access to treatme...,Neutral,$0.00,NaN
4,House Bill,1733.0,An Act improving access to behavioral health s...,Neutral,$0.00,NaN
5,Senate Bill,2984.0,An Act promoting a resilient health care syste...,Neutral,$0.00,NaN
6,House Bill,4134.0,An Act to improve health care by investing in ...,Neutral,$0.00,NaN
7,House Bill,1131.0,An Act relative to patient centered access to ...,Neutral,$0.00,NaN
8,Senate Bill,2769.0,An Act Putting Patients First,Neutral,$0.00,NaN
9,House Bill,4888.0,An Act Putting Patients First,Neutral,$0.00,NaN


In [234]:
dfs[2]

,0,1,2
0,Records prior to registration year 2005 are no...,Search data is collected from registration rec...,NaN
1,Records prior to registration year 2005 are no...,Your comments are welcome: submit feedback,Your comments are welcome: submit feedback


In [188]:
some_tables

[<tr style="vertical-align: top">
 <td style="width: 25%; vertical-align: top">
 <span class="p1" id="ContentPlaceHolder1_ERegistrationInfoReview1_Label14">Authorizing Officer name</span>
 </td>
 <td style="width: 75%">
 <span class="p2" id="ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityAuthorizingOfficerFirstName">John</span>
 <span class="p2" id="ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityAuthorizingOfficerMiddleName"></span>
 <span class="p2" id="ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityAuthorizingOfficerLastName">Bartley</span>
 </td>
 </tr>,
 <tr style="vertical-align: top">
 <td>
 <span class="p1" id="ContentPlaceHolder1_ERegistrationInfoReview1_Label13">Title</span>
 </td>
 <td>
 <span class="p2" id="ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityTitle">Principal</span>
 </td>
 </tr>,
 <tr style="vertical-align: top">
 <td>
 <span class="p1" id="ContentPlaceHolder1_ERegistrationInfoReview1_Label12">Business name</span>
 </td>
 <td>
 <span c

In [5]:
import random
html_list = html20 + html05
test_htmls = random.choices(html_list, k=10)

In [146]:
import pandas as pd
import re

class LobbyingDataPage:
    def __init__(self, html):
        self.html = html
        self.tables = {}
        self.dfs = pd.read_html(html)
        self.soup = bs(self.html, 'html.parser')
        for data in self.soup(['style','script']):
            data.decompose()
        self.cleanstring = " ".join(self.soup.stripped_strings)
        if 'An Error Occurred' in str(self.dfs[0][0]):
            #end this. Set default values?
            pass
        else:
            self.is_entity = 'Entity' in self.dfs[4][0][2]
            self.get_date_range()
            self.get_header()
            self.scrape_tables()

    def get_date_range(self):
        self.date_range = self.dfs[4][0][2].split('period:  ')[1]

    #Extracts the table of header info from the top of the page
    #TODO merge entites and lobbyists tables??
    # add Lobbyist/Entity column
    # fill out n/a fields with n/a's
    # entity / lobbyist
    # authorizing officer name / lobbyist name
    # ao title / na
    # business name / business name
    # address, citty, state, zip, country, phone / same
    # na / agent type

    def get_header(self):

        table = self.dfs[5][0:7].transpose()
        table.columns = table.iloc[0]
        table = table[1:]

        if self.is_entity:
            self.tables['Entities'] = table
        else:
            self.tables['Lobbyists'] = table
            
    def scrape_tables(self):
        for i in range(len(self.dfs[5:])):
            df_str = str(self.dfs[i])
            #ACTIVITIES TABLES
            if 'House / Senate' in self.dfs[i] and len(self.dfs[i]) == 1:
                self.get_activities(i)
            
            #CLIENT COMPENSATION
            if 'Client Compensation' in df_str and len(self.dfs[i]) == 2:
                self.get_compensation(i)

            #SALARIES
            if 'Salaries' in df_str and len(self.dfs[i].columns) == 2:
                self.get_salaries(i)

            #OPERATING EXPENSES

            #ENTERTAINMENT EXPENSES

            #ADDITIONAL EXPENSES

            #CAMPAIGN CONTRIBUTIONS

        #Strip whitespace from tables
        for key in self.tables.keys():
            self.tables[key] = self.tables[key].applymap(lambda x: re.sub('\s\s+', ' ',x).strip() if isinstance(x, str) else x)


    def get_activities(self, i):
        self.tables.setdefault('Activities', pd.DataFrame()) #Create table if it doesn't exist
        client = str(self.dfs[i-1][0][0]).split('Client:')[1].strip()
        if self.is_entity:
            lobbyist = self.dfs[i-2][0][0].split('Lobbyist:')[1].strip()
        else:
            lobbyist = str(self.tables['Lobbyists']['Lobbyist name'][1])
        table = self.dfs[i+1][:-1]
        table.insert(0, 'Client', client)
        table.insert(0, 'Lobbyist', lobbyist)
        table.insert(0, 'Date Range', self.date_range)
        self.tables['Activities'] = pd.concat( [self.tables['Activities'], table])

    def get_compensation(self, i):
        self.tables.setdefault('Compensation', pd.DataFrame())
        comp_str = self.dfs[i][0][1]
        data = re.findall(r'[\w\s\.&,]+\s\$[\d,\.]+', comp_str[11:])
        data = [d.split(" $") for d in data]
        data = [[d[0], float(d[1].replace(',',''))] for d in data if len(d) == 2]
        table = pd.DataFrame(data, columns = ['Name', 'Amount'])
        self.tables['Compensation'] = pd.concat( [self.tables['Compensation'], table[:-1]])

    def get_salaries(self, i):
        self.tables.setdefault('Salaries', pd.DataFrame())
        table = self.dfs[i][:-1]
        self.tables['Salaries'] = pd.concat( [self.tables['Salaries'], table])

    def get_operating_expenses(self, i):
        pass

    def get_entertainment_expenses(self, i):
        pass

    def get_additional_expenses(self, i):
        pass


    ## TODO this compiles fine, but is missing A LOT of tables!
    def get_campaign_contributions(self, i):
        self.tables.setdefault('Contributions', pd.DataFrame())
        data = self.dfs[i][0][5].split("  ")[1:-2]
        
        datenames = data[0::4] # WHY is it like this???
        dates = [datename.split()[0] for datename in datenames]
        names = [datename[11:] for datename in datenames]
        recipient_names = data[1::4]
        offices_sought = data[2::4]
        amounts = data[3::4]

        table = pd.DataFrame(list(zip(dates, names, recipient_names, offices_sought, amounts)), columns =['Date', 'Lobbyist Name', 'Recipient Name', 'Office Sought', 'Amount'])
        self.tables['Contributions'] = pd.concat( [self.tables['Contributions'], table])

    def clean_entry(entry):
        return re.sub("\s\s+", " ", entry)

    def fetch_tables(self):
        return self.tables
        
    def save(self):
        for table in self.tables.keys():
            self.write_data(f'../data/{table.replace(" ","_").lower()}.csv', self.tables[table])

    def write_data(self, file_path, dataframe):
        write = True
        if os.path.exists(file_path):
            # with open(file_path, mode = 'r', encoding = 'utf-8') as f:
            #     for line in f:
            #         if self.company_name in line and self.date_range in line:
            #             print('Data already present in ' + file_path)
            #             write = False
            #             break

            if write and type(dataframe) == pd.DataFrame:
                print('Saving data to ' + file_path)
                dataframe.to_csv(file_path, mode ='a+',header=(not os.path.exists(file_path)), index=False)

In [182]:
ldps = [LobbyingDataPage(html) for html in test_htmls]

NO DATA
NO DATA


IndexError: list index out of range

In [172]:
# for ldp in ldps:
#     print(ldp.fetch_tables().keys())

dfs = ldps[0].dfs
results = [df for df in dfs if 'House / Senate' in df]

results[0]

AttributeError: 'LobbyingDataPage' object has no attribute 'dfs'

In [21]:
all_tables = []
for html in test_htmls:
    all_tables.append(LobbyingDataPage(html).fetch_tables())

In [26]:
all_tables[0]['Entities']

,Authorizing Officer name,Title,Business name,Address,"City, state, zip code",Country,Phone
1,John Bartley,Principal,Beacon Hill Consulting Group,20 Ashburton Place,"Boston, MA 02108",USA,617-216-2060


In [24]:
all_tables[1]['Entities']

,Lobbyist name,Business name,Address,"City, state, zip code",Country,Agent type,Phone
1,Ellen Attaliades,Association of Developmental Disabilities Prov...,1671 Worcsester Road,"Framingham, MA 01710",USA,Both,508-422-7156


In [ ]:
# Header:
# Authorizing Officer name / Lobbyist name
# Title / NA
# Business name / Business name
# Address / Address
# City, state, zip code / Citym, state, zip code
# country / country
# NA / Agent Type

#TABLES:
# Entities
# Lobbyists
# Client Compensation
# Compensation/Salaries Paid
# Activities, Bill Numbers and Titles
# Operating Expenses
# Meals, Travel, and Entertainment Expenses
# Additional Expenses
# Campaign Contributions

In [186]:
import re
html = html_list[1]

soup = bs(html,'html.parser')
for data in soup(['style','script']):
    data.decompose()
html_str = " ".join(soup.stripped_strings)

is_entity = bool(soup.find('span', {'id': 'ContentPlaceHolder1_ERegistrationInfoReview1_lblEntityCompany'}))

#header
def get_header(html_str):
    if is_entity:
        date_range = re.search('reporting period :(.+)Authorizing ',html_str).group(1)
        print(date_range.strip())
        name = re.search('Officer name(.+)Title ',html_str).group(1)
        print(name.strip())
        title = re.search('Title(.*)Business name',html_str).group(1)
        print(title.strip())
        business_name = re.search('Business name(.*)Address',html_str).group(1)
        print(business_name.strip())
        address = re.search('Address(.*)City',html_str).group(1)
        print(address.strip())
        city_state_zip = re.search('zip code(.*),(.*) (.*) Country',html_str)
        city = city_state_zip.group(1).strip()
        state = city_state_zip.group(2).strip()
        zipcode = city_state_zip.group(3).strip()
        print(city)
        print(state)
        print(zipcode)
        country = re.search('Country(.*)Phone',html_str).group(1)
        print(country.strip())
        phone = re.search('Phone (\d\d\d.\d\d\d.\d\d\d\d)',html_str).group(1)
        print(phone.strip())
    else:
        date_range = re.search('reporting period :(.+)Lobbyist ',html_str).group(1)
        print(date_range.strip())
        name = re.search('Lobbyist name(.+)Business ',html_str).group(1)
        print(name.strip())
        agent_type = re.search('Agent type(.+)Phone ',html_str).group(1)
        print(agent_type.strip())
        business_name = re.search('Business name(.*)Address',html_str).group(1)
        print(business_name.strip())
        address = re.search('Address(.*)City',html_str).group(1)
        print(address.strip())
        city_state_zip = re.search('zip code(.*),(.*) (.*) Country',html_str)
        city = city_state_zip.group(1).strip()
        state = city_state_zip.group(2).strip()
        zipcode = city_state_zip.group(3).strip()
        print(city)
        print(state)
        print(zipcode)
        country = re.search('Country(.*)Agent type',html_str).group(1)
        print(country.strip())
        phone = re.search('Phone (\d\d\d.\d\d\d.\d\d\d\d)',html_str).group(1)
        print(phone)


#print(html_str)
re.findall('Amount Direct business association (.*) Total amount',html_str)


["Executive Other Efforts to grow and expand the Commonwealth's early college/college-in-high-s chool programs. Support $0.00 N/A House Bill 2 An Act making appropriations for the fiscal year 2021 for the maintenance of the departments, boards, commissions, institutions, and certain activities of the commonwealth, for interest, sinking fund, and serial bond requirements, and for certain permanent improvements Support $0.00 N/A Total amount $0.00 Lobbyist: Hugh Richard Jones, III Client: The Education Trust Inc House / Senate Bill Number or Agency Name Bill title or activity Agent position Amount Direct business association House Bill 2 An Act making appropriations for the fiscal year 2021 for the maintenance of the departments, boards, commissions, institutions, and certain activities of the commonwealth, for interest, sinking fund, and serial bond requirements, and for certain permanent improvements Support $0.00 N/A Executive Other Efforts to address the Commonwealth's education equi

In [184]:
html_str

"Lobbyist Public Search Home Directions Contact Us Lobbyist Public Search Records prior to registration year 2005 are not available online. Contact the Lobbyist Division here for assistance. Search data is collected from registration records and disclosure reports \r\n        submitted by lobbyists, lobbying entities, and their clients. Massachusetts General Law requires any party providing or retaining \r\n        lobbying services to register annually, and submit disclosure reports twice a year. Your comments are welcome: submit feedback Disclosure reporting details Back to search detail summary Lobbyist disclosure reporting period : 01/01/2020 - 06/30/2020 Lobbyist name Lisa C. Adams Business name Massachusetts Municipal Association Address One Winthrop Square City, state, zip code Boston, MA 02110 Country USA Agent type Both Phone 617-426-7272 Operating Expenses No operating expenses were filed for this disclosure reporting period. Activities, Bill Numbers and Titles Client: Massac